In [1]:
%%time
# coding: utf-8

# In[1]:


# !ls -hl|grep csv
import pandas as pd
import numpy as np
import os
from time import time, ctime

from sklearn.metrics import accuracy_score, classification_report, classification, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit

dim=lambda *x: [i.shape for i in x]


# In[1]:


import os
import tensorflow as tf
from tensorflow.python.client import device_lib

CPU times: user 798 ms, sys: 2.35 s, total: 3.15 s
Wall time: 3.74 s


In [2]:
%%time
# from keras import backend as K

# config = tf.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.Session(config=config)

# K.set_session(sess)

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(device_lib.list_local_devices())


# In[4]:


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator



import os



# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())


# In[5]:


df = pd.read_csv('./ready.csv')
print (df.shape)
X=df.values

# In[6]:
y=pd.read_csv('./label_ready.csv')['label'].values

# In[7]:




# In[8]:




# In[11]:


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=7)


# In[12]:


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print (dim(X_train,X_test, y_train, y_test))


# In[13]:


classifier = Sequential()
# first hidden layer
classifier.add(Dense(units = 256, 
                     input_dim=4491,
                     kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.5))
# second hidden layer
classifier.add(Dense(units = 128,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.4))
# second hidden layer
classifier.add(Dense(units = 64,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.3))
# thrid hidden layer
classifier.add(Dense(units = 48,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.2))
# thrid hidden layer
classifier.add(Dense(units = 32,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 24,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 256,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.5))
# # ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))
# compiling the ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy', metrics= ['accuracy'])
print (classifier.summary())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8128725444864438587
]


Using TensorFlow backend.


(49260, 4491)
[(34482, 4491), (14778, 4491), (34482,), (14778,)]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1149952   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (De

In [3]:



# In[14]:


from keras.callbacks import Callback, EarlyStopping
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=1)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


# In[15]:


stopping = EarlyStopping(monitor='acc', min_delta=0,
                              patience=6, verbose=1, mode='auto')


# In[16]:


# fitting ANN with training set
classifier.fit(X_train, y_train, 
               batch_size=128, epochs=128,
               validation_data=(X_test, y_test),
          callbacks=[TestCallback((X_test, y_test))])
#           callbacks=[stopping])
    


# In[53]:


y_pred = classifier.predict(X_test)
y_pred = (y_pred> 0.5)

print (accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print (cm)



Train on 34482 samples, validate on 14778 samples
Epoch 1/128
14432/14778 [============================>.] - ETA: 0s
Testing loss: 0.460357750429, acc: 0.81979970222

34482/34482 [==============================] - 4s - loss: 0.4870 - acc: 0.8193 - val_loss: 0.4604 - val_acc: 0.8198
Epoch 2/128
14432/14778 [============================>.] - ETA: 0s
Testing loss: 0.464991271072, acc: 0.819123020666

34482/34482 [==============================] - 4s - loss: 0.4397 - acc: 0.8203 - val_loss: 0.4650 - val_acc: 0.8191
Epoch 3/128
14528/14778 [============================>.] - ETA: 0s
Testing loss: 0.484206529065, acc: 0.817431316782

34482/34482 [==============================] - 4s - loss: 0.4096 - acc: 0.8311 - val_loss: 0.4842 - val_acc: 0.8174
Epoch 4/128
14528/14778 [============================>.] - ETA: 0s
Testing loss: 0.518139389709, acc: 0.816483962607

34482/34482 [==============================] - 4s - loss: 0.3865 - acc: 0.8416 - val_loss: 0.5181 - val_acc: 0.8165
Epoch 5/128
142

34482/34482 [==============================] - 4s - loss: 0.0451 - acc: 0.9843 - val_loss: 1.5951 - val_acc: 0.7969
Epoch 36/128
14144/14778 [===========================>..] - ETA: 0s
Testing loss: 1.72979044461, acc: 0.799093246678

34482/34482 [==============================] - 4s - loss: 0.0429 - acc: 0.9851 - val_loss: 1.7298 - val_acc: 0.7991
Epoch 37/128
14240/14778 [===========================>..] - ETA: 0s
Testing loss: 1.65757415444, acc: 0.798754905901

34482/34482 [==============================] - 4s - loss: 0.0409 - acc: 0.9865 - val_loss: 1.6576 - val_acc: 0.7988
Epoch 38/128
14208/14778 [===========================>..] - ETA: 0s
Testing loss: 1.74219272905, acc: 0.79638652052

34482/34482 [==============================] - 4s - loss: 0.0398 - acc: 0.9861 - val_loss: 1.7422 - val_acc: 0.7964
Epoch 39/128
14336/14778 [============================>.] - ETA: 0s
Testing loss: 1.55399085521, acc: 0.800378941686

34482/34482 [==============================] - 4s - loss: 0.0376 

34482/34482 [==============================] - 4s - loss: 0.0206 - acc: 0.9929 - val_loss: 1.8503 - val_acc: 0.7995
Epoch 71/128
14528/14778 [============================>.] - ETA: 0s
Testing loss: 1.84189195384, acc: 0.799634591921

34482/34482 [==============================] - 4s - loss: 0.0237 - acc: 0.9931 - val_loss: 1.8419 - val_acc: 0.7996
Epoch 72/128
14464/14778 [============================>.] - ETA: 0s
Testing loss: 2.09629488247, acc: 0.801732304785

34482/34482 [==============================] - 4s - loss: 0.0217 - acc: 0.9936 - val_loss: 2.0963 - val_acc: 0.8017
Epoch 73/128
14304/14778 [============================>.] - ETA: 0s
Testing loss: 1.86580693644, acc: 0.797333874638

34482/34482 [==============================] - 4s - loss: 0.0207 - acc: 0.9932 - val_loss: 1.8658 - val_acc: 0.7973
Epoch 74/128
14432/14778 [============================>.] - ETA: 0s
Testing loss: 1.85958548301, acc: 0.803424008678

34482/34482 [==============================] - 4s - loss: 0.0215

34482/34482 [==============================] - 4s - loss: 0.0177 - acc: 0.9948 - val_loss: 1.9511 - val_acc: 0.7969
Epoch 106/128
14240/14778 [===========================>..] - ETA: 0s
Testing loss: 2.10208840223, acc: 0.798484233279

34482/34482 [==============================] - 4s - loss: 0.0175 - acc: 0.9945 - val_loss: 2.1021 - val_acc: 0.7985
Epoch 107/128
14208/14778 [===========================>..] - ETA: 0s
Testing loss: 2.03028201565, acc: 0.79584517522

34482/34482 [==============================] - 4s - loss: 0.0173 - acc: 0.9943 - val_loss: 2.0303 - val_acc: 0.7958
Epoch 108/128
14272/14778 [===========================>..] - ETA: 0s
Testing loss: 1.87056672926, acc: 0.795709838909

34482/34482 [==============================] - 4s - loss: 0.0204 - acc: 0.9940 - val_loss: 1.8706 - val_acc: 0.7957
Epoch 109/128
14240/14778 [===========================>..] - ETA: 0s
Testing loss: 1.94152379096, acc: 0.796386520463

34482/34482 [==============================] - 4s - loss: 0.0